In [25]:

import pandas as pd
from gensim.models import Word2Vec
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import *

from utils import vectorize, average_word2vec

In [2]:
train = pd.read_csv('./datasets/ex_train.csv')
test = pd.read_csv('./datasets/ex_test.csv')
valid = pd.read_csv('./datasets/ex_validation.csv')

In [3]:
train.head()  # 어차피 category는 `economy` or `IT과학`이다.

,date,category,press,title,document,link,summary
0,2022-07-03 17:14:37,economy,YTN,추경호 중기 수출지원 총력 무역금융 40조 확대,앵커 정부가 올해 하반기 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 ...,https://n.news.naver.com/mnews/article/052/000...,"올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록한 가운데, ..."
1,2022-07-04 08:07:12,economy,아시아경제,해산물·주류 무제한 인터컨티넨탈 뷔페 여름 한정 페스타 연다,문어 랍스터 대게 갑오징어 새우 소라 등 해산물 활용 미국식 해물찜 시푸드 보일 준...,https://n.news.naver.com/mnews/article/277/000...,인터엑스 1층 뷔페 레스토랑 브래서리는 오는 6일부터 8월31일까지 쿨 섬머 페스타...
2,2022-07-05 18:01:01,IT과학,동아사이언스,한국인 생활 변화시킨 의사 마지막까지 영향력 컸던 과학자 이호왕 고려대 명예교수 소천,한탄바이러스 발견 노벨상 유력 후보로 자주 거론 한국을 대표하는 의학자이자 미생물학...,https://n.news.naver.com/mnews/article/584/000...,이 이호왕 고려대 명예교수는 바이러스의 병원체와 진단법 백신까지 모두 개발한 한국을...
3,2022-07-01 08:51:12,economy,뉴시스,에디슨이노 이승훈 제이스페이스 대표 사내이사 선임,기사내용 요약 우주발사체 사업 본격화 서울 뉴시스 김경택 기자 에디슨이노가 우주발사...,https://n.news.naver.com/mnews/article/003/001...,에디슨이노는 1일 임시주주총회를 통해 사명을 이노시스 로 변경하고 이승훈 제이스페이...
4,2022-07-01 16:11:01,economy,머니투데이,SK바사 해외 사업 조직 개편… 글로벌 탑티어 기업으로 성장 박차,SK바이오사이언스가 글로벌 사업의 고도화를 위해 조직 개편을 단행했다. SK바이오사...,https://n.news.naver.com/mnews/article/008/000...,SK바이오사이언스가 글로벌 사업의 고도화를 위해 기존 해외사업개발실을 백신사업뿐만 ...


In [4]:
train.drop(['date', 'press', 'link'], axis=1, inplace=True)
test.drop(['date', 'press', 'link'], axis=1, inplace=True)
valid.drop(['date', 'press', 'link'], axis=1, inplace=True)
train.head()

,category,title,document,summary
0,economy,추경호 중기 수출지원 총력 무역금융 40조 확대,앵커 정부가 올해 하반기 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 ...,"올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록한 가운데, ..."
1,economy,해산물·주류 무제한 인터컨티넨탈 뷔페 여름 한정 페스타 연다,문어 랍스터 대게 갑오징어 새우 소라 등 해산물 활용 미국식 해물찜 시푸드 보일 준...,인터엑스 1층 뷔페 레스토랑 브래서리는 오는 6일부터 8월31일까지 쿨 섬머 페스타...
2,IT과학,한국인 생활 변화시킨 의사 마지막까지 영향력 컸던 과학자 이호왕 고려대 명예교수 소천,한탄바이러스 발견 노벨상 유력 후보로 자주 거론 한국을 대표하는 의학자이자 미생물학...,이 이호왕 고려대 명예교수는 바이러스의 병원체와 진단법 백신까지 모두 개발한 한국을...
3,economy,에디슨이노 이승훈 제이스페이스 대표 사내이사 선임,기사내용 요약 우주발사체 사업 본격화 서울 뉴시스 김경택 기자 에디슨이노가 우주발사...,에디슨이노는 1일 임시주주총회를 통해 사명을 이노시스 로 변경하고 이승훈 제이스페이...
4,economy,SK바사 해외 사업 조직 개편… 글로벌 탑티어 기업으로 성장 박차,SK바이오사이언스가 글로벌 사업의 고도화를 위해 조직 개편을 단행했다. SK바이오사...,SK바이오사이언스가 글로벌 사업의 고도화를 위해 기존 해외사업개발실을 백신사업뿐만 ...


In [5]:
train.category.value_counts()

category
economy    17088
IT과학        5106
Name: count, dtype: int64

In [6]:
from sklearn.preprocessing import LabelEncoder

# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(train['category'])
encoded_labels  #, train.category

array([1, 1, 0, ..., 1, 0, 0])

In [9]:
# from utils import clean_punc
# 
# refined_document = [clean_punc(txt) for txt in train.document]
#
# okt = Okt()
# 
# tokenized_data = []
# ready = tqdm(refined_document)
# 
# for sentence in ready:
#     tokenized_sentence = okt.morphs(sentence, stem=True)  # 토큰화
#     stopwords_removed_sentence = [word for word in tokenized_sentence if not word in STOP_WORDS]  # 불용어 제거
#     tokenized_data.append(stopwords_removed_sentence)

# 시간 소요가 심하므로 `tokenized_data`는 txt 파일에 저장함

In [10]:
# import json
# 
# with open('tokenized_data.json', 'w') as f:
#     json.dump(tokenized_data, f)

In [11]:
import json

# 불러오기
with open('tokenized_data.json', 'r') as f:
    tokenized_data = json.load(f)

In [12]:
tokenized_data

[['앵커',
  '정부',
  '올해',
  '하반기',
  '경제',
  '버팀목',
  '수출',
  '확대',
  '위해',
  '총력',
  '기울이다',
  '특히',
  '수출',
  '중소기업',
  '물류',
  '난',
  '해소',
  '위해',
  '무역',
  '금융',
  '규모',
  '40조',
  '확대',
  '물류',
  '비',
  '지원',
  '임시',
  '선박',
  '투입',
  '추진',
  '류환',
  '홍',
  '기자',
  '보도',
  '기자',
  '수출',
  '은',
  '최고',
  '실적',
  '보이다',
  '수입',
  '액',
  '급증',
  '올해',
  '상반기',
  '우리나라',
  '무역수지',
  '는',
  '역대',
  '최악',
  '103억',
  '달러',
  '적자',
  '기록',
  '정부',
  '수출',
  '확대',
  '총력',
  '기울이다',
  '은',
  '원자재',
  '가격',
  '상승',
  '대외',
  '리스크',
  '가중',
  '상황',
  '수출',
  '증가',
  '세',
  '지속',
  '이야말로',
  '한국',
  '경제',
  '회복',
  '위',
  '열쇠',
  '라고',
  '보다',
  '이다',
  '추경호',
  '경제부총리',
  '겸',
  '기획재정부',
  '장관',
  '정부',
  '는',
  '경제',
  '성장',
  '엔진',
  '수출',
  '높다',
  '증가',
  '세',
  '지속',
  '총력',
  '다',
  '물류',
  '부담',
  '증가',
  '원자재',
  '가격',
  '상승',
  '가중',
  '대외',
  '리스크',
  '적극',
  '대응',
  '특히',
  '중소기업',
  '중견',
  '기업',
  '수출',
  '지원',
  '위해',
  '무역',
  '금융',
  '규모',
  '연초',
  '목표',
  '보다',
  '40조',
  '늘

In [13]:
model = Word2Vec(sentences=tokenized_data, vector_size=100, window=5, min_count=1, workers=4)

In [14]:
doc_vectors = [average_word2vec(doc, model) for doc in tokenized_data]

In [30]:
# Scale vectors for MNB
scaler = MinMaxScaler()
scaled_vectors = scaler.fit_transform(doc_vectors)

In [33]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(scaled_vectors, encoded_labels, test_size=0.2, random_state=24)

# 별로 효과가 없음.
# smote = SMOTE(random_state=42)
# X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

classifier = MultinomialNB()
classifier.fit(X_train, y_train)

MultinomialNB()

In [34]:
from sklearn.metrics import classification_report

# Evaluate
predictions = classifier.predict(X_test)
print(classification_report(y_test, predictions))

accuracy_train = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy_train)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1016
           1       0.77      1.00      0.87      3423

    accuracy                           0.77      4439
   macro avg       0.39      0.50      0.44      4439
weighted avg       0.59      0.77      0.67      4439

Accuracy: 0.771119621536382


/Users/kreimben/Repository/NewsCategoriser/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/kreimben/Repository/NewsCategoriser/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/kreimben/Repository/NewsCategoriser/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf

# 주어진 데이터(`ex_test`, `ex_validation`)로 예측을 해보자 

In [18]:
valid_vector = vectorize(valid.document)

100%|██████████| 2466/2466 [01:00<00:00, 40.46it/s]


In [37]:
from sklearn.preprocessing import LabelEncoder

# Encode labels
label_encoder = LabelEncoder()
encoded_valid = label_encoder.fit_transform(valid['category'])

In [38]:
predictions = classifier.predict(valid_vector)
print(classification_report(encoded_valid, predictions))

accuracy_valid = accuracy_score(encoded_valid, predictions)
print("Accuracy:", accuracy_valid)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       593
           1       0.76      1.00      0.86      1873

    accuracy                           0.76      2466
   macro avg       0.38      0.50      0.43      2466
weighted avg       0.58      0.76      0.66      2466

Accuracy: 0.759529602595296


/Users/kreimben/Repository/NewsCategoriser/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/kreimben/Repository/NewsCategoriser/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/kreimben/Repository/NewsCategoriser/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf

In [21]:
test_vector = vectorize(test.document)

100%|██████████| 2740/2740 [01:01<00:00, 44.42it/s]


In [39]:
from sklearn.preprocessing import LabelEncoder

# Encode labels
label_encoder = LabelEncoder()
encoded_test = label_encoder.fit_transform(test['category'])

In [40]:
predictions = classifier.predict(test_vector)
print(classification_report(encoded_test, predictions))

accuracy_test = accuracy_score(encoded_test, predictions)
print("Accuracy:", accuracy_test)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       652
           1       0.76      1.00      0.86      2088

    accuracy                           0.76      2740
   macro avg       0.38      0.50      0.43      2740
weighted avg       0.58      0.76      0.66      2740

Accuracy: 0.762043795620438


/Users/kreimben/Repository/NewsCategoriser/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/kreimben/Repository/NewsCategoriser/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/kreimben/Repository/NewsCategoriser/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf